In [212]:
#importing libraries:
import requests
import itertools
import pandas as pd
from bs4 import BeautifulSoup
from pandas import DataFrame as df 
import re

In [213]:
URLs = pd.read_csv('CBI_URLteste.csv')
URLs_list = pd.DataFrame(URLs)
# print (URL_column)
count_before = len(URLs_list)
print ('The database has', count_before,  'URL listed')

The database has 1 URL listed


In [214]:
URLs_list_cleaned = URLs_list.drop_duplicates(subset=['URLs'])
URLs_list_cleaned.count()
URL_column = URLs_list_cleaned['URLs']

In [215]:
Loc_ID = pd.read_excel('CBI_Locations_ID.xlsx')
Location_ID = pd.DataFrame(Loc_ID)

In [216]:
Job_Titles_DB = pd.read_excel('Job_Titles_CBI_to_SunLife.xlsx')
JT_CBI_SunLife = pd.DataFrame(Job_Titles_DB)
# JT_CBI_SunLife

In [217]:
Services_DB = pd.read_excel('Services_CBI_to_SunLife.xlsx')
Services_CBI_SunLife = pd.DataFrame(Services_DB)
# Services_CBI_SunLife

In [218]:
def Location_Name():
    Locationportlet = soup.find_all('div', attrs = {'class' : 'p-locationaddress2-address'})
    # print(Locationportlet)
    for i in Locationportlet:
        Location = soup.find('h2').text
    # print (Location)
    return Location

#Extracting Services
def Services():
    Services_source = []
    for i in soup.find_all('div', attrs = {'class', 'row-fluid p-locationservices-bizservices-row'}):
        Services_source.append (i.get_text(separator=';'))

    if len(Services_source) == 2:
        Services_source_1 = Services_source [0] + Services_source [1]
#         print (Services_source_1)
    else:
        Services_source_1 = ''.join(Services_source)

    # print (Services_source_1)

    Services_1 = re.sub('; ;', ';', Services_source_1)
    # print (Services_1)
    Services_2 = re.sub('; ;', ';', Services_1)
    # print (Services_2)
    Services_3 = re.sub(';  ;', ';', Services_2)
    # print (Services_3)
    Services_4 = re.sub('; ;', ';', Services_3)
    # print (Services_4)

    Services = Services_4[Services_4.find(';')+1:Services_4.rfind(';')]
    return Services

Services()

'Acupuncture/IMS;Concussion;Ergonomics;Kinesiology;MVA Rehabilitation;Physiotherapy;Pre/Post-Surgical Rehabilitation;Psychology;Return to Work Planning;Social Work;Vestibular Rehabilitation;Occupational Therapy & Physiotherapy;Sports Injury'

In [219]:
 ############################ Loop in URL's and Creating the list to put in a Dataframe #####################################

Final_list_1 = []


 ########################################## Main For Loop on th list of URL's ################################################
for i in URL_column:
    #Reading the URL's (i) on the document:
    r = requests.get (i)
    soup = BeautifulSoup (r.text, 'html.parser')
    # print (r.text)
    General = soup.findAll( 'div' , attrs = {'class' : 'item text-center'})
#     print (General)

    
 ################################### creating the list with the clinic information ##########################################  
    row = [Location_Name()]




    
 ################################  Crossing the location with the Location_ID database #######################################
    Location_list = []
    Location_ID_list = []
    Location_list.append(Location_Name())
    Location_dataframe = pd.DataFrame(Location_list)
    Location_dataframe.columns = ['Location']
#     print (Location_dataframe)
    ID_Location = Location_dataframe.merge (Location_ID, left_on='Location', right_on='Location', how='left')
    ID_Location.set_index('Location', inplace = True , drop = True)
#     print (ID_Location)
    ID_location_final = ID_Location.loc [Location_Name(), 'Location_ID']
#     print (ID_location_final)
    ID_location_final_string = str(ID_location_final)
#     print (ID_location_final_string)


 ################################### Loop on the team portlet and creating the list ##########################################
    
    row_2_source = []

    Headshots = soup.find_all('div', attrs = {'class' : 'item text-center'})
#     print (Headshots)
    
    if Headshots == []:
        
        row_2_source = [None] * 160
        row_2 = row_2_source
        
    
    else:    
        for i in Headshots:
            
            img = i.find('img', alt=True)
            string1 = img['alt']
            First =  string1.find(',')
            Second = string1.find(',',First+1)
            Name = string1[ : First]
            First_Name = Name [ : Name.find(' ')]
            Last_Name = Name [Name.rfind(' ')+1 : ]
            File_Name = First_Name + '-' + Last_Name + '-' + ID_location_final_string
            
            
            Job = string1[First+2 : Second]
            Job_list = []
            Job_list.append (Job)
            Job_Titles_df = pd.DataFrame(Job_list)
            Job_Titles_df.columns = ['CBI_Titles']
#             print (Job_Titles_df)
            Job_Title_SunLife = Job_Titles_df.merge (JT_CBI_SunLife, left_on='CBI_Titles', right_on='CBI_Titles', how='left')
            Job_Title_SunLife.set_index('CBI_Titles', inplace = True , drop = True)
            Job_Title_final = Job_Title_SunLife.loc [Job, 'SunLife_Titles']
             
        
#             print (Job_Title_SunLife)
            Bio = string1[Second+2 : ]
            if Job_Title_final != 'REVISE':
                row_2_source.append (First_Name)
                row_2_source.append (Last_Name)
                row_2_source.append (File_Name)
                row_2_source.append (Job_Title_final)
                Missing_Columns = 160 - len(row_2_source) # creating the list with the clinic information
                Empty_rows = [None] * Missing_Columns
                row_2 = row_2_source + Empty_rows
            else:
                continue
    
        
#     print (row)
#     print (row_2)

            
 ################################### creating the list with the clinic information ##########################################
            



    final_row = row + row_2
    Final_list_1.append (final_row)

# Final_list_1
# Location_Name()

In [220]:
Dataframe = pd.DataFrame(Final_list_1)
Dataframe.columns = ['Location',                  
                     'P1_First_Name', 'P1_Last_Name', 'P1_Image_File_Name', 'P1_Specialty',
                     'P2_First_Name', 'P2_Last_Name', 'P2_Image_File_Name', 'P2_Specialty',
                     'P3_First_Name', 'P3_Last_Name', 'P3_Image_File_Name', 'P3_Specialty',
                     'P4_First_Name', 'P4_Last_Name', 'P4_Image_File_Name', 'P4_Specialty', 
                     'P5_First_Name', 'P5_Last_Name', 'P5_Image_File_Name', 'P5_Specialty', 
                     'P6_First_Name', 'P6_Last_Name', 'P6_Image_File_Name', 'P6_Specialty', 
                     'P7_First_Name', 'P7_Last_Name', 'P7_Image_File_Name', 'P7_Specialty', 
                     'P8_First_Name', 'P8_Last_Name', 'P8_Image_File_Name', 'P8_Specialty',
                     'P9_First_Name', 'P9_Last_Name', 'P9_Image_File_Name', 'P9_Specialty', 
                     'P10_First_Name', 'P10_Last_Name', 'P10_Image_File_Name', 'P10_Specialty', 
                     'P11_First_Name', 'P11_Last_Name', 'P11_Image_File_Name', 'P11_Specialty', 
                     'P12_First_Name', 'P12_Last_Name', 'P12_Image_File_Name', 'P12_Specialty', 
                     'P13_First_Name', 'P13_Last_Name', 'P13_Image_File_Name', 'P13_Specialty', 
                     'P14_First_Name', 'P14_Last_Name', 'P14_Image_File_Name', 'P14_Specialty', 
                     'P15_First_Name', 'P15_Last_Name', 'P15_Image_File_Name', 'P15_Specialty', 
                     'P16_First_Name', 'P16_Last_Name', 'P16_Image_File_Name', 'P16_Specialty', 
                     'P17_First_Name', 'P17_Last_Name', 'P17_Image_File_Name', 'P17_Specialty', 
                     'P18_First_Name', 'P18_Last_Name', 'P18_Image_File_Name', 'P18_Specialty', 
                     'P19_First_Name', 'P19_Last_Name', 'P19_Image_File_Name', 'P19_Specialty', 
                     'P20_First_Name', 'P20_Last_Name', 'P20_Image_File_Name', 'P20_Specialty', 
                     'P21_First_Name', 'P21_Last_Name', 'P21_Image_File_Name', 'P21_Specialty', 
                     'P22_First_Name', 'P22_Last_Name', 'P22_Image_File_Name', 'P22_Specialty', 
                     'P23_First_Name', 'P23_Last_Name', 'P23_Image_File_Name', 'P23_Specialty', 
                     'P24_First_Name', 'P24_Last_Name', 'P24_Image_File_Name', 'P24_Specialty', 
                     'P25_First_Name', 'P25_Last_Name', 'P25_Image_File_Name', 'P25_Specialty',
                     'P26_First_Name', 'P26_Last_Name', 'P26_Image_File_Name', 'P26_Specialty',
                     'P27_First_Name', 'P27_Last_Name', 'P27_Image_File_Name', 'P27_Specialty',
                     'P28_First_Name', 'P28_Last_Name', 'P28_Image_File_Name', 'P28_Specialty',
                     'P29_First_Name', 'P29_Last_Name', 'P29_Image_File_Name', 'P29_Specialty',
                     'P30_First_Name', 'P30_Last_Name', 'P30_Image_File_Name', 'P30_Specialty',
                     'P31_First_Name', 'P31_Last_Name', 'P31_Image_File_Name', 'P31_Specialty',
                     'P32_First_Name', 'P32_Last_Name', 'P32_Image_File_Name', 'P32_Specialty',
                     'P33_First_Name', 'P33_Last_Name', 'P33_Image_File_Name', 'P33_Specialty',
                     'P34_First_Name', 'P34_Last_Name', 'P34_Image_File_Name', 'P34_Specialty',
                     'P35_First_Name', 'P35_Last_Name', 'P35_Image_File_Name', 'P35_Specialty',
                     'P36_First_Name', 'P36_Last_Name', 'P36_Image_File_Name', 'P36_Specialty',
                     'P37_First_Name', 'P37_Last_Name', 'P37_Image_File_Name', 'P37_Specialty',
                     'P38_First_Name', 'P38_Last_Name', 'P38_Image_File_Name', 'P38_Specialty',
                     'P39_First_Name', 'P39_Last_Name', 'P39_Image_File_Name', 'P39_Specialty',
                     'P40_First_Name', 'P40_Last_Name', 'P40_Image_File_Name', 'P40_Specialty',]
Dataframe

,Location,P1_First_Name,P1_Last_Name,P1_Image_File_Name,P1_Specialty,P2_First_Name,P2_Last_Name,P2_Image_File_Name,P2_Specialty,P3_First_Name,...,P38_Image_File_Name,P38_Specialty,P39_First_Name,P39_Last_Name,P39_Image_File_Name,P39_Specialty,P40_First_Name,P40_Last_Name,P40_Image_File_Name,P40_Specialty
0,CBI Health Centre - Lloydminster,Kim,Hartman,Kim-Hartman-64,Physiotherapist,Sandabria,McPhie,Sandabria-McPhie-64,Physiotherapist,Scott,...,None,None,None,None,None,None,None,None,None,None


In [221]:
# Database = Services_CBI_SunLife

Services_1 = Services().split (";")
Services_list = []

Services_Dataframe = pd.DataFrame(Services_1)
Services_Dataframe.columns = ['Services_CBI']
Services_Dataframe.set_index('Services_CBI', inplace = True , drop = True)
# Job_Title_final = Job_Title_SunLife.loc [Job, 'SunLife_Titles']
Services_Dataframe

Services_SunLife_Crossed = Services_Dataframe.merge (Services_CBI_SunLife, left_on='Services_CBI', right_on='CBI_Services', how='left')
Cleaned_dataframe = Services_SunLife_Crossed.loc[Services_SunLife_Crossed['Sun_Life_Services']!= 'REVISE']
Services_to_use = Cleaned_dataframe ['Sun_Life_Services']
Services_to_use_1 = Services_to_use.drop_duplicates()
Services_to_use_1

for i in Services_to_use_1:
    Services_list.append(i)


print(Services_list)
    
# Services_Dataframe = Services.merge (JT_CBI_SunLife, left_on='CBI_Titles', right_on='CBI_Titles', how='left')
# Job_Title_SunLife.set_index('CBI_Titles', inplace = True , drop = True)

['Acupuncture', 'Concussion therapy', 'Ergonomics', 'Physiotherapy', 'Pre/post-surgical rehabilitation', 'Psychology']


In [238]:
def Services():
    Services_source = []
    Service_list_1 = []
    for i in soup.find_all('div', attrs = {'class', 'row-fluid p-locationservices-bizservices-row'}):
        Services_source.append (i.get_text(separator=';'))

    if len(Services_source) == 2:
        Services_source_1 = Services_source [0] + Services_source [1]
#         print (Services_source_1)
    else:
        Services_source_1 = ''.join(Services_source)

    # print (Services_source_1)

    Services_1 = re.sub('; ;', ';', Services_source_1)
    # print (Services_1)
    Services_2 = re.sub('; ;', ';', Services_1)
    # print (Services_2)
    Services_3 = re.sub(';  ;', ';', Services_2)
    # print (Services_3)
    Services_4 = re.sub('; ;', ';', Services_3)
    # print (Services_4)
    Services = Services_4[Services_4.find(';')+1:Services_4.rfind(';')]
    
    Services_1 = Services.split (";")
    Services_list = []
    Services_Dataframe = pd.DataFrame(Services_1)
    Services_Dataframe.columns = ['Services_CBI']
    Services_Dataframe.set_index('Services_CBI', inplace = True , drop = True)
    # Job_Title_final = Job_Title_SunLife.loc [Job, 'SunLife_Titles']
    Services_Dataframe

    Services_SunLife_Crossed = Services_Dataframe.merge (Services_CBI_SunLife, left_on='Services_CBI', right_on='CBI_Services', how='left')
    Cleaned_dataframe = Services_SunLife_Crossed.loc[Services_SunLife_Crossed['Sun_Life_Services']!= 'REVISE']
    Services_to_use = Cleaned_dataframe ['Sun_Life_Services']
    Services_to_use_1 = Services_to_use.drop_duplicates()
    Services_to_use_1
    
    for i in Services_to_use_1:
        Services_list.append(i)
#     print (Services_list)
    
    Services_list_1 = ";".join(Services_list)
#         for i in Services_list:
#             Service_list_1.append(i)
        
    return Services_list_1

Services()

'Acupuncture;Concussion therapy;Ergonomics;Physiotherapy;Pre/post-surgical rehabilitation;Psychology'

In [143]:
Services_Dataframe.to_excel('services_list.xlsx')

In [145]:
Services_CBI_SunLife

,CBI_Services,Sun_Life_Services
0,Acupuncture/IMS,Acupuncture
1,Advanced Foot Care,REVISE
2,Behaviour Therapy,REVISE
3,Chiropractic,Chiropractic
4,Cognitive Rehabilitation,Cognitive rehabilitation
5,Concussion,Concussion therapy
6,Custom Orthotics/Bracing & Splinting,Bracing & splinting
7,Disability Management,REVISE
8,Driving Assessment,REVISE
9,Ergonomics,Ergonomics
